<a href="https://colab.research.google.com/github/5amessi/YOLO-For-Face-Detection/blob/master/YoloV2_Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
!wget http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip
!wget http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/example/Submission_example.zip
import zipfile
path = "/content/drive/My Drive/WIDER_train.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("")
zip_ref.close()
path = "/content/wider_face_split.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("")
zip_ref.close()

--2020-02-01 23:08:19--  http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip
Resolving mmlab.ie.cuhk.edu.hk (mmlab.ie.cuhk.edu.hk)... 137.189.99.12
Connecting to mmlab.ie.cuhk.edu.hk (mmlab.ie.cuhk.edu.hk)|137.189.99.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3591642 (3.4M) [application/zip]
Saving to: ‘wider_face_split.zip’

wider_face_split.zi 100%[===================>]   3.42M  --.-KB/s    in 0.1s    

2020-02-01 23:08:20 (29.6 MB/s) - ‘wider_face_split.zip’ saved [3591642/3591642]

--2020-02-01 23:08:23--  http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/example/Submission_example.zip
Resolving mmlab.ie.cuhk.edu.hk (mmlab.ie.cuhk.edu.hk)... 137.189.99.12
Connecting to mmlab.ie.cuhk.edu.hk (mmlab.ie.cuhk.edu.hk)|137.189.99.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 624 [application/zip]
Saving to: ‘Submission_example.zip’

Submission_example. 100%[===================>]     

In [0]:
#import libs
import os
import os.path as path
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import *
from keras.layers import *
from keras.utils import *
from keras import backend as K
from PIL import Image
import pandas as pd
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split
from keras.applications import VGG19 ,inception_v3
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf

In [0]:
baseMapNum = 32
weight_decay = 1e-4
# ANCHORS = [0.57273, 0.677385]
ANCHORS = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
BOX = 5
GRID_H = GRID_W = 13
BATCH_SIZE = 4
IMAGE_H, IMAGE_W = 440,440
OBJ_THRESHOLD    = 0.5
NMS_THRESHOLD    = 0.5
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [0]:
def TransformCord(bx,img):
  Nx,Ny, Nw, Nh = bx[0]/img.shape[1], bx[1]/img.shape[0], bx[2]/img.shape[1], bx[3]/img.shape[0]
  Nx = Nx + Nw/2
  Ny = Ny + Nh/2
  Nw = bx[2] / img.shape[1]
  Nh = bx[3] / img.shape[0]
  return Nx,Ny, Nw, Nh


In [0]:
def AdjustCord(bx,cx,cy):
  cell_W = (1/GRID_W)
  cell_H = (1/GRID_H)
  Nx = (bx[0] - cx) / cell_W
  Ny = (bx[1] - cy) / cell_H
  return Nx,Ny


In [0]:
def ReAdjustCord(bx,imgW,imgH,cx,cy):
  cell_W = (1/GRID_W)
  cell_H = (1/GRID_W)
  
  Nx = (bx[0] * cell_W) + cx 
  Ny = (bx[1] * cell_H) + cy
  
  Nx -= bx[2]/2 
  Ny -= bx[3]/2 
  
  Nx *= imgW
  Ny *= imgH
  
  Nw = bx[2] * imgW
  Nh = bx[3] * imgH
  return Nx,Ny, Nw, Nh

In [0]:
annotations = open('/content/wider_face_split/wider_face_train_bbx_gt.txt')
lines = annotations.readlines()
numOfLines = len(lines)

def GenerateBatch(i,END):
  Label = []
  Images = []
  while i<END and i < numOfLines:
    numOfFace= int(lines[i+1])
    if(int(numOfFace) <= GRID_H**2):
      imgName=str(lines[i].rstrip('\n'))
      path = "/content/WIDER_train/images/"+imgName
      img = cv2.imread(path)
      Images.append(cv2.resize(img,(IMAGE_H, IMAGE_W),interpolation = cv2.INTER_AREA))
      
      visted = [0]*3000
      liney = []
      for y in range(GRID_H):
        linex = []
        for x in range(GRID_W):
          check = False
          StartIdx = i+2
          
          for k in range(StartIdx,StartIdx+numOfFace):
            if not visted[k-StartIdx]:
              bx = list(map(int,lines[k].split(' ')[:-1]))
              Nx,Ny, Nw, Nh = TransformCord(bx,img)#Scale range to (0-1)
              if((Nx > (x / GRID_W) and Nx <= ((x+1)/GRID_W))  and (Ny > (y / GRID_H) and Ny <= ((y+1)/GRID_H))):#Check if face exist in this cell
                Nx,Ny = AdjustCord([Nx,Ny] , x/GRID_W ,  y/GRID_H)
                # print(bx)
                # print(Nx,Ny, Nw, Nh)
                # print(ReAdjustCord([Nx,Ny, Nw, Nh], img.shape[1],img.shape[0], x/GRID_W, y/GRID_H))
                # print("")
                linex.append([[Nx,Ny,Nw,Nh,1],[Nx,Ny,Nw,Nh,1],[Nx,Ny,Nw,Nh,1],[Nx,Ny,Nw,Nh,1],[Nx,Ny,Nw,Nh,1]])
                check = True
                visted[k-StartIdx] = 1
                break
       
          if check == False:
            linex.append([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]])
        liney.append(linex)
          
      Label.append(liney)
    i+= max(numOfFace,1)+2
    
  return i , np.array(Images) , np.array(Label)

In [0]:
#building classifer 
def C_N_N():
    model = Sequential()
    model.add(Conv2D(8*baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),
                     input_shape=(IMAGE_H, IMAGE_W, 3),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(8*baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(4 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(4 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))  

    model.add(Conv2D(2 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(2 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))  
    
    model.add(Conv2D(2 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.1))  
   
    model.add(Conv2D(baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(BOX * (4 + 1), (1,1), strides=(1,1)))
    model.add(Reshape((GRID_H, GRID_W,BOX, 4 + 1)))

    return model

In [8]:
#compile it
model = C_N_N()
model.summary()


/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 440, 440, 256)     7168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 440, 440, 256)     1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 440, 440, 256)     590080    
_________________________________________________________________
batch_normalization_2 (Batch (None, 440, 440, 256)     1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 220, 220, 256)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 220, 220, 256)     0         

In [0]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    print("Mask",np.shape(mask_shape))

    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    print("cell_x",np.shape(cell_x))
    cell_y = cell_x
    print("cell_y",np.shape(cell_y))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 5, 1])
    print("cell_grid",np.shape(cell_grid))

    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    print("coord_mask",np.shape(coord_mask))
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])
    
    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    
    ### adjust class probabilities
    
    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
    
    # ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       
    print("pred0",np.shape(pred_mins),np.shape(true_mins),np.shape(pred_maxes),np.shape(true_maxes))
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    print("iou_scores",np.shape(iou_scores))
    true_box_conf = iou_scores * y_true[..., 4]
    print("true_box_conf",np.shape(true_box_conf))

    # true_box_conf = y_true[..., 4]
    
    ### adjust class probabilities
    
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    
    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = y_true[..., 0:2]
    true_wh = y_true[..., 2:4]
    print("true_xy",np.shape(true_xy),np.shape(true_wh))
    
    
    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half
    
    # pred_xy = tf.expand_dims(pred_box_xy, 4)
    # pred_wh = tf.expand_dims(pred_box_wh, 4)
    # pred_xy = pred_box_xy
    # pred_wh = pred_box_wh
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half    
   
    print("pred",np.shape(pred_mins),np.shape(true_mins),np.shape(pred_maxes),np.shape(true_maxes))

    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    print("0")
    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]
    print("1")
    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    print("iou_scores",np.shape(iou_scores))
    print("2")
    best_ious = tf.reduce_max(iou_scores, axis=3,keepdims=True)
    print("best_ious",np.shape(best_ious))
    print("2.5")
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * NO_OBJECT_SCALE
    print("3")
    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * OBJECT_SCALE
        
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    seen = tf.assign_add(seen, 1.)
    
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])
    
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    
    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    
    loss = loss_xy + loss_wh + loss_conf
    
    nb_true_box = tf.reduce_sum(y_true[..., 4])
    nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

    """
    Debugging code
    """    
    current_recall = nb_pred_box/(nb_true_box + 1e-6)
    total_recall = tf.assign_add(total_recall, current_recall) 

    loss = tf.Print(loss, [tf.zeros((1))], message='Dummy Line \t', summarize=1000)
    loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
    loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
    loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
    loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
    loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=1000)
    loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=1000)
    
    return loss

In [64]:
model.compile(loss=custom_loss, optimizer=keras.optimizers.adam())

Mask (4,)
cell_x (1, 13, 13, 1, 1)
cell_y (1, 13, 13, 1, 1)
cell_grid (4, 13, 13, 5, 2)
coord_mask (?, ?, ?, ?)
pred0 (4, 13, 13, 5, 2) (?, ?, ?, ?, ?) (4, 13, 13, 5, 2) (?, ?, ?, ?, ?)
iou_scores (4, 13, 13, 5)
true_box_conf (4, 13, 13, 5)
true_xy (?, ?, ?, ?, ?) (?, ?, ?, ?, ?)
pred (4, 13, 13, 5, 2) (?, ?, ?, ?, ?) (4, 13, 13, 5, 2) (?, ?, ?, ?, ?)
0
1
iou_scores (4, 13, 13, 5)
2
best_ious (4, 13, 13, 1)
2.5
3


In [0]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    print("Mask",np.shape(mask_shape))

    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    print("cell_x",np.shape(cell_x))
    cell_y = cell_x
    print("cell_y",np.shape(cell_y))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 5, 1])
    print("cell_grid",np.shape(cell_grid))

    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    print("coord_mask",np.shape(coord_mask))
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])    
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    """
    Adjust ground truth
    """
    true_box_xy = y_true[..., 0:2] 
    true_box_wh = y_true[..., 2:4] 
    true_box_conf = y_true[..., 4]
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    
    seen = tf.assign_add(seen, 1.)
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES),
                   lambda: [true_box_xy + (0.5) * no_boxes_mask, 
                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2])* no_boxes_mask, tf.ones_like(coord_mask)],
                   lambda: [true_box_xy, true_box_wh, coord_mask])
    
 
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    # nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))

    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(tf.sqrt(true_box_wh)-tf.sqrt(pred_box_wh))     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf))
    loss = loss_xy + loss_wh + loss_conf
    
    return loss

In [67]:
i=0
i, x, y = GenerateBatch(i,500)
print(i)
x_train, x_test, y_train ,y_test = train_test_split(x, y, test_size=0.2, random_state=50)
print(np.shape(y_train))
# y_train = np.reshape(y_train,(len(x_train),13, 13, BOX, 5))
# print(np.shape(y_train))

516
(13, 13, 13, 5, 5)


In [68]:
model.fit(x_train, y_train, epochs=50, batch_size=BATCH_SIZE, validation_data=(x_test,y_test), verbose=1)
model.save_weights("/content/drive/My Drive/WiderFaceweights2.h5")

Train on 13 samples, validate on 4 samples
Epoch 1/50
13/13 [==============================] - 2s 149ms/step - loss: 136.5662 - val_loss: 2068431.6250
Epoch 2/50
13/13 [==============================] - 1s 108ms/step - loss: 316.8434 - val_loss: 33.4964
Epoch 3/50
13/13 [==============================] - 1s 107ms/step - loss: 70.5530 - val_loss: 32.9947
Epoch 4/50
13/13 [==============================] - 1s 108ms/step - loss: 69.7531 - val_loss: 33.0097
Epoch 5/50
13/13 [==============================] - 1s 108ms/step - loss: 69.8137 - val_loss: 33.1308
Epoch 6/50
13/13 [==============================] - 1s 108ms/step - loss: 61.0399 - val_loss: 33.2666
Epoch 7/50
13/13 [==============================] - 1s 108ms/step - loss: 82.9978 - val_loss: 33.1725
Epoch 8/50
13/13 [==============================] - 1s 108ms/step - loss: 77.9098 - val_loss: nan
Epoch 9/50
13/13 [==============================] - 1s 108ms/step - loss: 118.2222 - val_loss: nan
Epoch 10/50
13/13 [====================

In [0]:
print(np.exp(0.5))
print(np.log(np.exp(0.05)))

In [0]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [0]:
_, tempx, ty = GenerateBatch(0,1000)
img = tempx[0]
true_box_wh = ty[..., 2:4]
print(np.shape(true_box_wh))
i = ty[0].reshape((GRID_H * GRID_W,5))[67]
print(i)
# Nx,Ny, Nw, Nh = ReAdjustCord(i[0]*(8/GRID_W),i[1]*(5/GRID_H), i[2], i[3],img)
# cv2.rectangle(img, (int(Nx),int(Ny)), (int(Nw)+int(Nx),int(Nh)+int(Ny)), (255,0,0)) 
# cv2_imshow(img)
# print(Nx,Ny, Nw, Nh)
tempy = model.predict(tempx)
i = tempy[0].reshape((GRID_H * GRID_W,5))[67]
Nx,Ny, Nw, Nh = sigmoid(i[0]),sigmoid(i[1]), np.exp(i[2]), np.exp(i[3])
print(Nx,Ny, Nw, Nh)
Nx,Ny, Nw, Nh = ReAdjustCord(Nx*(8/GRID_W),Ny*(5/GRID_H), Nw, Nh,img)
print(Nx,Ny, Nw, Nh)
cv2.rectangle(img, (int(Nx),int(Ny)), (int(Nw)+int(Nx),int(Nh)+int(Ny)), (255,0,0)) 
cv2_imshow(img)

In [0]:
for i in range(GRID_H * GRID_W):
  tt = ty[0].reshape((GRID_H * GRID_W,5))[i]
  if(tt[4] > 0.5):
    print(tt)
    print(i)

In [0]:
_, tempx, ty = GenerateBatch(0,1000)
tempy = model.predict(tempx)
img = tempx[0]
print(np.shape(tempy[0]))
for idx,i in enumerate(ty[0].reshape((9,5))):
    if(float(i[4]) > 0.5):
      print("ssss",i,idx)
      Nx,Ny, Nw, Nh = sigmoid(i[0]),sigmoid(i[1]), i[2], i[3]
      print(Nx,Ny, Nw, Nh)
      Nx,Ny, Nw, Nh = ReAdjustCord(i[0],i[1], i[2], i[3],img)
      print(Nx,Ny, Nw, Nh)
      cv2.rectangle(img, (int(Nx),int(Ny)), (int(Nw)+int(Nx),int(Nh)+int(Ny)), (255,0,0)) 
      cv2_imshow(img)


In [0]:
model.save_weights("/content/drive/My Drive/Face Detection/weights.h5")

In [0]:
# predicted = model.predict(x)
# predicted = np.reshape(predicted,(1,3*3,5))
# for i in predicted[0]:
#   if(float(i[0]) > 0.5):
#     print(i)
# print(np.sum(predicted,axis = 1))
# print(np.sum(np.reshape(y,(1,3*3,5)),axis = 1))